In [1]:
# Fitted Q iteration (Expected SARSA)

In [2]:
# policy_train_path = '../continuous/dqn_normal/dqn_normal_actions_train.p'
# policy_val_path = '../continuous/dqn_normal/dqn_normal_actions_val.p'
# policy_test_path = '../continuous/dqn_normal/dqn_normal_actions_test.p'

In [4]:
import numpy as np
import math
import os
import random
import pandas as pd
from pandas import DataFrame
try:
    import cPickle as pickle
except ImportError:
    import _pickle as pickle
import copy

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [7]:
with open('../data/state_features.txt') as f:
    state_features = f.read().split()
print (state_features)
print (len(state_features))

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission', 'bloc']
48


In [8]:
df = pd.read_csv('../data/rl_train_data_final_cont.csv')

In [9]:
df.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,mortality_90d,Weight_kg,...,median_dose_vaso,max_dose_vaso,input_total_tev,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev,vaso_input,iv_input,reward
0,0.000000,3,7245052800,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.797351,0.939195,0.589916,0.750908,0.554500,0.0,4.0,0.125000
1,0.222560,3,7245067200,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.831780,0.934543,0.674384,0.819589,0.580033,0.0,4.0,0.657321
2,0.356608,3,7245081600,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.833222,0.656575,0.765423,0.939329,0.555033,0.0,2.0,1.367788
3,0.452837,3,7245096000,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.834033,0.603831,0.783597,0.847073,0.545700,0.0,2.0,1.199099
4,0.527957,3,7245110400,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.834836,0.603831,0.794059,0.811583,0.539533,0.0,2.0,1.057596


In [10]:
val_df = pd.read_csv('../data/rl_val_data_final_cont.csv')

In [11]:
test_df = pd.read_csv('../data/rl_test_data_final_cont.csv')

In [12]:
# # Here we load the actions for the policy we want to evaluate into the relevant dataframes
# policy_train = pickle.load(open(policy_train_path, "rb" ))
# policy_test = pickle.load(open(policy_test_path, "rb" ))
# policy_val = pickle.load(open(policy_val_path, "rb" ))

# df['policy'] = np.array(policy_train)
# test_df['policy'] = np.array(policy_test)
# val_df['policy'] = np.array(policy_val)


save_dir = './MixNN_EM/'
train_actions = np.load(save_dir + "cond_lik_train_dr.npy")
val_actions = np.load(save_dir + "cond_lik_val_dr.npy")
test_actions = np.load(save_dir + "cond_lik_test_dr.npy")

mix_comp_eval = 3 # SET THIS ACCORDINGLY
train_actions = [np.array(list(i)) for i in train_actions[:,:,mix_comp_eval]]
val_actions = [np.array(list(i)) for i in val_actions[:,:,mix_comp_eval]]
test_actions = [np.array(list(i)) for i in test_actions[:,:,mix_comp_eval]]

df['policy'] = train_actions
val_df['policy'] = val_actions
test_df['policy'] = test_actions

In [13]:
fq_state_terms = copy.deepcopy(state_features)
fq_state_terms.append('iv_input')
fq_state_terms.append('vaso_input')

In [14]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [15]:
action_tuple = []
for iv in range(5):
    for vaso in range(5):
        action_tuple.append([iv,vaso])
action_tuple = np.array(action_tuple)

In [16]:
# first define all the (s,a) pairs
# then create (s', a') for each possible next action. This is of size (num_points*25)
# call predict on each of these 
# reshape into (-1,25): (num_points, 25)
# multiply elementwise by prob_actions
# reduce sum along 1 axis to get expected Q values

In [17]:
def make_s_a(df_in):
    cur_s_a = df_in[fq_state_terms].values
    return cur_s_a

In [18]:
# save which indices of the df are terminal
def find_term_indices(df_in):
    term = []
    df_local = df_in.reset_index()
    for count, i in enumerate(df_local.index):
        if count % 10000 == 0 and count > 0:
            print ("find_term_indices, step %d" % count)
        if i != df_local.index[-1]:
            if df_local.loc[i, 'icustayid'] != df_local.loc[i+1, 'icustayid']:
                term.append(count)
        else:
            # last state is the terminal one
            term.append(count)
    return np.array(term)       

In [19]:
# construct s', a' for all a in action space
def make_s_next_a_next(df_in):
    df_local = df_in.reset_index()
    s_next = df_local[fq_state_terms].values
    initial_shape = s_next.shape
    
    # add padding at the end
    s_next = np.append(s_next, [np.zeros(initial_shape[1])], axis=0)
    # remove first elem
    s_next = s_next[1:]  
    
    # tile in 0 dimension to get the different actions
    s_next_all_actions = np.repeat(s_next, 25, axis=0)
    
    
    act_tup_tiled = np.tile(action_tuple, [initial_shape[0],1])
    
    s_next_all_actions[:,-2:] = act_tup_tiled  
    
    return s_next_all_actions
    

In [20]:
def make_prob_actions(df_in):
    df_local = df_in.reset_index()
    prob = df_local['policy'].values
    prob = np.vstack([arr for arr in prob])
    prob = np.append(prob, [np.zeros(25)], axis=0)
    prob = prob[1:]
    return prob

In [24]:
# call predict on s_next_all_actions
# weight by prob_actions
# zero out the terminal ones
def make_exp_next_q(s_next_all_actions, prob_actions, term_indices, reg):
    if reg is None:
        return np.zeros(len(s_next_all_actions)//25)
    else:
        next_q = reg.predict(s_next_all_actions)
        next_q = next_q.reshape([-1,25])
        exp_q = np.sum(prob_actions*next_q, axis=1)
        exp_q[term_indices] = 0
        return exp_q       

In [25]:
# prepare data
# train
train_X = make_s_a(df)
train_term_indices = find_term_indices(df)
train_s_next_a_next = make_s_next_a_next(df)
train_prob_actions = make_prob_actions(df)

# val
val_X = make_s_a(val_df)
val_term_indices = find_term_indices(val_df)
val_s_next_a_next = make_s_next_a_next(val_df)
val_prob_actions = make_prob_actions(val_df)

# test
test_X = make_s_a(test_df)
test_term_indices = find_term_indices(test_df)
test_s_next_a_next = make_s_next_a_next(test_df)
test_prob_actions = make_prob_actions(test_df)

reg = None


find_term_indices, step 10000
find_term_indices, step 20000
find_term_indices, step 30000
find_term_indices, step 40000
find_term_indices, step 50000
find_term_indices, step 60000
find_term_indices, step 70000
find_term_indices, step 80000
find_term_indices, step 90000
find_term_indices, step 100000
find_term_indices, step 110000
find_term_indices, step 120000
find_term_indices, step 130000
find_term_indices, step 140000
find_term_indices, step 150000
find_term_indices, step 160000
find_term_indices, step 10000
find_term_indices, step 20000
find_term_indices, step 10000
find_term_indices, step 20000
find_term_indices, step 30000
find_term_indices, step 40000


In [54]:
# New training loop
from sklearn.linear_model import SGDRegressor
# from sklearn.ensemble import RandomForestRegressor

# reg = None

num_iters = 20

gamma = 0.99

for i in range(num_iters):
    next_q = make_exp_next_q(train_s_next_a_next, train_prob_actions, train_term_indices, reg)
    train_y = df['reward'] + gamma * next_q
    
    # create new reg object, and then call fit on train_X and train_y
    reg = SGDRegressor(max_iter=500)
#     reg = RandomForestRegressor(n_estimators=500, verbose=0, n_jobs=-1)
    
    reg.fit(train_X, train_y)

    val_next_q  = make_exp_next_q(val_s_next_a_next, val_prob_actions, val_term_indices, reg)
    val_y = val_df['reward'] + gamma * val_next_q
    pred = reg.predict(val_X)
    mean_error = np.mean((pred - val_y)**2)
    print ("Mean on val set is %f, mse is %f" % (np.mean(pred),mean_error))
    

Mean on val set is 9.851984, mse is 6.470908
Mean on val set is 9.902630, mse is 6.468600
Mean on val set is 9.999157, mse is 6.460223
Mean on val set is 9.998372, mse is 6.458193
Mean on val set is 9.928467, mse is 6.466437
Mean on val set is 9.894042, mse is 6.465933
Mean on val set is 9.872244, mse is 6.464393
Mean on val set is 9.929383, mse is 6.461929
Mean on val set is 10.011618, mse is 6.454670
Mean on val set is 10.046761, mse is 6.457443
Mean on val set is 10.072494, mse is 6.458645
Mean on val set is 9.976383, mse is 6.473614
Mean on val set is 9.912758, mse is 6.483040
Mean on val set is 9.806124, mse is 6.494962
Mean on val set is 9.737536, mse is 6.495072
Mean on val set is 9.664712, mse is 6.500106
Mean on val set is 9.770194, mse is 6.483073
Mean on val set is 9.848758, mse is 6.477232
Mean on val set is 9.806058, mse is 6.486038
Mean on val set is 9.853844, mse is 6.472068


In [55]:
val_q = reg.predict(val_X)

In [64]:
list(zip(val_df['reward'].values[val_q>0],val_q[val_q>0]))

[(0.74485620705799993, 7.4453346060150096),
 (0.40318489575700001, 9.0797233432229731),
 (-0.025000000000000001, 9.6173451802248255),
 (-0.224916749916, 8.8253216369638352),
 (-0.34991674991599997, 7.9612661623623433),
 (0.27508325008400003, 9.9346322645554874),
 (-0.224916749916, 11.808009536006892),
 (-0.12491674991599999, 11.138421355828928),
 (-0.12491674991599999, 10.867703151028266),
 (-0.72491674991600008, 11.35201823989771),
 (15.0, 11.681620913260398),
 (-0.647088045399, 9.762542045406212),
 (-1.1470880454000001, 8.9646466969907266),
 (-0.79708804539900002, 8.9357284227601994),
 (-0.897088045399, 8.7844611122079002),
 (-0.647088045399, 8.110028818253701),
 (-0.897088045399, 7.7966529638353581),
 (-0.647088045399, 7.625388550904125),
 (-1.0220880454000001, 7.8300042576277562),
 (-1.1470880454000001, 7.1288890532885949),
 (-0.79708804539900002, 8.7231665805235323),
 (15.0, 8.9961710551652612),
 (0.5, 14.55532366459979),
 (-0.055766803446300001, 16.088003351747226),
 (-0.05576680

In [79]:
# get final preds for val and test sets
val_q = reg.predict(val_X)
test_q = reg.predict(test_X)

np.save('val_q_learned_policy.npy', val_q)
np.save('test_q_learned_policy.npy', test_q)

In [80]:
from sklearn.externals import joblib
joblib.dump(reg, 'lr_model.pkl')

['lr_model.pkl']